# Exploring the regional response to a warmer world

This notebook allows you to explore the regional response to a warmer world at a series of global **warming levels** (changes in global mean temperature relative to the historical baseline): 1.5˚, 2˚, 3˚, and 4˚C. Warming levels can be used to explore future possible trajectories of any climate variable of interest for a given increase in global mean temperature.

Because warming levels are defined based on amount of global mean temperature change, they can be used to compare possible outcomes across multiple scenarios or model simulations. The same total warming benchmark may be reached at different times in different simulations. Warming levels are often used in international policy discussions, such as the [Paris Agreement](https://unfccc.int/process-and-meetings/the-paris-agreement/the-paris-agreement) to limit warming to 2˚C.

For a given warming level, we will then use this tool to:

- Examine the range of possibility in maps of regional changes in temperature (or any variable of interest) across climate models dynamically downscaled to-date

- Map day-of-year/time-of-day combinations with especially large increases in temperature, and overlay the locations of power plants to help identify vulnerable assets

## Step 0: Setup

Import libraries needed.

In [1]:
import panel as pn
pn.extension()

import climakitae as ck

ModuleNotFoundError: No module named 'climakitae'

Additionally, get set up to make the computing go faster by executing the following cell. Learn more about dask and see some common [troubleshooting tips on our FAQ page](https://analytics.cal-adapt.org/docs/faq/).

In [ ]:
from dask_gateway import GatewayCluster
cluster = GatewayCluster()
cluster.adapt(minimum=0, maximum=16)
client = cluster.get_client()
cluster

Launch the tools in the [climakitae](https://github.com/cal-adapt/climakitae) library:

In [ ]:
app = ck.Application()

## Step 1: Explore data

Launch a panel to view localized projections under varying levels of warming. First, select a warming level, variable, and spatial area of interest. The right panel will show projected changes in global mean temperature -- use the drop down menu to visualize when simulations of a scenario of interest will reach the specified global warming level. Scenarios shown are Shared Socioeconomic Pathways ([SSPs](https://www.sciencedirect.com/science/article/pii/S0959378016300681)): ranging from low (SSP 1-1.9) to high (SSP 5-8.5) emissions trajectories. 

Simulations of high emissions scenarios will generally reach a given warming level faster than simulations of lower emissions scenarios.

The bottom panel shows regional responses of the variable of interest for the selected warming level for each individual model simulation. You can also see an average, minimum, and maximum regional response across simulations. 

To learn more about the data available on the Analytics Engine, [see our data catalog](https://analytics.cal-adapt.org/data/). 

In [ ]:
app.explore.warming_levels()

## Step 2: Application

Prepare a map of the changes for a day and time when the largest temperature increases are projected, and overlay the locations of California [power plants](https://cecgis-caenergy.opendata.arcgis.com/datasets/CAEnergy::california-power-plants/explore?location=37.174065,-118.394256,6.93) and [substations](https://cecgis-caenergy.opendata.arcgis.com/datasets/CAEnergy::california-electric-substations/explore?location=37.225884,-119.175779,6.93) on the map to identify vulnerable assets.

We'll start by retrieving the same data shown above, making sure that it is hourly time-resolution:

In [ ]:
app.selections.timescale = 'hourly'

In [ ]:
my_data = app.retrieve()
my_data

Here we select a particular day of the year and time of day, based on inspection of the typical meteorological year difference plot in the explore panel in Step 1:

In [ ]:
one_day = my_data.isel(time=(my_data.time.dt.dayofyear == 216))
one_hour = one_day.isel(time=(one_day.time.dt.hour == 23))
one_hour

And format the information to be plotted as a difference from historical for a given warming level:

In [ ]:
from climakitae.warming_levels import get_anomaly_data

my_gwl = 3.0  # 3˚ global warming
plots = get_anomaly_data(one_hour, warmlevel=my_gwl)
plots.name = 'degrees warmer'
plots

Force the data to be computed and ready to plot. This may take a minute.

In [ ]:
plots = plots.compute()

Load some data that contains locations of power plants:

In [ ]:
import geopandas as gpd

# URLs to shapefiles for example point data from CEC (power plants and substations available)
URLs = {
    'power_plants' : "https://opendata.arcgis.com/api/v3/datasets/4a702cd67be24ae7ab8173423a768e1b_0/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1",
    'substations' : "https://cecgis-caenergy.opendata.arcgis.com/datasets/CAEnergy::california-electric-substations.geojson?outSR=%7B%22latestWkid%22%3A3857%2C%22wkid%22%3A102100%7D"
}

# Download the desired data
power_plants = gpd.read_file(URLs['power_plants']).rename(columns = {'Lon_WGS84':'lon', 'Lat_WGS84':'lat'})

Finally, prepare the map, and display it with power plant locations overlaid. 

In [ ]:
import cartopy.crs as ccrs

gridded = plots.plot.pcolormesh('lon','lat', col='simulation',
                     transform=ccrs.PlateCarree(),subplot_kws={'projection': ccrs.Orthographic(-100,40)},
                         figsize=[15,5],cmap='Reds')
for ax in gridded.axes.flat:
    ax.coastlines()
    ax.scatter(power_plants.lon, power_plants.lat, transform=ccrs.PlateCarree(),s=0.2,c='k')

## Step 3: Export

To export, first pick a format from the dropdown menu.
- We recommend NetCDF, which will work with any number of variables and dimensions in your dataset
- CSV and GeoTIFF can only be used for data arrays with one variable
- CSV works best for up to 2-dimensional data (e.g., lon x lat), and will be compressed and exported with a separate metadata file
- GeoTIFF can accept 3 dimensions total: 
    - X and Y dimensions are required
    - The third dimension is flexible and will be a "band" in the file: time, simulation, or scenario could go here
    - Metadata will be accessible as "tags" in the .tif

In [ ]:
app.export_as()

Next, write in the object you wish to export and your desired filename (in single or double quotation marks).

In [ ]:
app.export_dataset(plots,'my_filename')

## Step 4: Clean up

Lastly, close the client and cluster to free up any computing resources for next time. 

In [ ]:
client.close()
cluster.shutdown()